# Rank Clustering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import ast
from datetime import date
from copy import deepcopy
from functools import lru_cache

In [2]:
df = pd.read_csv('Data//owid-covid-data.csv')

In [3]:
#Filtering for countries using min population and removing the continent samples
dfPopThreshold = df[(df.population >= 1000000) & (df.continent.isnull() == False) & (df.date >= '2020-01-22')].copy()
countries = list(dfPopThreshold.location.unique())

In [4]:
faultyDataCountries = ['Turkmenistan', 'Puerto Rico', 'North Korea']
for country in faultyDataCountries:
    index = list(countries).index(country)
    countries.pop(index)

In [5]:
def daysFromStart(measureDate):
    year, month, day = measureDate.split('-')
    return (date(int(year),int(month),int(day)) - date(2020,1,22)).days

def plotCases(df, country, var = 'new_cases'):
    countryDF = df[df.location == country].copy()
    plt.plot(countryDF.date, countryDF[var] / countryDF['population'])
    
def getTimeSeries(df, country):
    countryDF = df[df.location == country].copy()
    return countryDF['new_cases'] / countryDF['population']

In [6]:
def imputateAndSmooth(df, column, country, method = 'linear', window = 7):
    countryDF = df[df['location'] == country].copy()
    countryDF.date = (countryDF.date).apply(daysFromStart)
    countryDF[column].interpolate(method = method, order = 2, inplace = True)
    
    #Smoothing
    countryDF[column] = countryDF[column].rolling(window = window).mean()[window-1:]
    
    #Replaces preceding values with zero with the assumption
    #That this was before cases arrived / testing began
    countryDF[column] = countryDF[column].replace(np.nan, 0)
    
    return countryDF

In [7]:
def finalizeData(df, var = 'new_cases', window = 14):
    newDF = pd.DataFrame()
    for country in countries:
        newDF = pd.concat([newDF,imputateAndSmooth(df,var,country,window = window)], axis = 0)
    return newDF

In [8]:
finalDF = finalizeData(dfPopThreshold)

In [9]:
def createCountryTimeSeries(df, countries):
    timeSeriesList = []
    for country in countries:
        timeSeries = []
        countryDF = df[df.location == country]
        try:
            startDate = countryDF.iloc[1,3] - 1
        except:
            print(country)
        endDate = countryDF.iloc[-1,3]
        timeSeriesList.append([0 for i in range(startDate)] + list(countryDF.new_cases / countryDF.population))
    return timeSeriesList
        

In [10]:
newCaseTimeSeries = createCountryTimeSeries(finalDF, countries)

#Creating normalized (local) lists of time series
globalMax = -math.inf
globalMin = math.inf
newCaseTimeSeriesNormalizedLocal = []
for i in range(len(newCaseTimeSeries)):
    maxVal = max(newCaseTimeSeries[i])
    if maxVal > globalMax: globalMax = maxVal
    minVal = min(newCaseTimeSeries[i])
    if minVal < globalMin: globalMin = minVal
    ts = []
    for val in newCaseTimeSeries[i]:
        ts.append((val - minVal) / (maxVal - minVal))
    newCaseTimeSeriesNormalizedLocal.append(ts)

#Creating normalized (global) lists of time series
newCaseTimeSeriesNormalizedGlobal = []
for i in range(len(newCaseTimeSeries)):
    ts = []
    for val in newCaseTimeSeries[i]:
        ts.append((val - globalMin) / (globalMax - globalMin))
    newCaseTimeSeriesNormalizedGlobal.append(ts)

In [13]:
def convertTimeSeriesToRank(TSList, method = 'dense'):
    newTSList = []
    for TS in TSList:
        TSSeries = pd.Series(TS)
        TSSeries = TSSeries.rank(method=method)
        newTSList.append(list(TSSeries))
    return newTSList

In [12]:
rankedNewCaseTimeSeries = convertTimeSeriesToRank(newCaseTimeSeries)

In [14]:
minRankedNewCaseTimeSeries = convertTimeSeriesToRank(newCaseTimeSeries, method = 'min')

In [25]:
for ts in range(len(minRankedNewCaseTimeSeries)):
    if max(minRankedNewCaseTimeSeries[ts]) != 846:
        print(countries[ts])

Bosnia and Herzegovina
Burundi
Chad
El Salvador
Equatorial Guinea
Gabon
Hungary
Liberia
Libya
Nicaragua
Paraguay
Tanzania
Venezuela


In [20]:
countries[100]

'Nicaragua'

# Clustering

In [30]:
import random
import re

In [31]:
def TSDist(ts1, ts2):
    """
    Average Geometric Distance (Root mean square distance).
    Assumes series' of equal length
    """
    squaredDistSum = 0
    for i in range(len(ts1)):
        squaredDistSum += (ts1[i] - ts2[i])**2
    return math.sqrt(squaredDistSum) / len(ts1)

In [32]:
def SWMatchEQ(ts1, ts2, maxShiftPerc = 20):
    """
    Sliding window time series matching given two time series' of equal length
    """
    maxTimeShift = int(len(ts1) * (maxShiftPerc / 100))
    
    #Positive minDistShift denotes the shift of the first time series
    minDistShift = 0
    minDist = TSDist(ts1, ts2)
    
    for i in range(maxTimeShift):
        dist = TSDist(ts1[i+1:], ts2[:-(i+1)])
        if dist < minDist:
            minDist = dist
            minDistShift = i + 1
    for i in range(maxTimeShift):
        dist = TSDist(ts2[i+1:], ts1[:-(i+1)])
        if dist < minDist:
            minDist = dist
            minDistShift = -i - 1
            
    return minDist, minDistShift

In [33]:
def plotShifts(ts1, ts2, shift, yMin = -3, yMax = 3):
    if shift != 0:
        aMod = ts1[abs(shift):] if shift < 0 else ts1[:-shift]
        bMod = ts2[:shift] if shift < 0 else ts2[shift:]
        #plt.plot([x for x in range(len(a))], a)
        #plt.plot([x for x in range(len(b))], b)
        plt.plot([x for x in range(len(aMod))], aMod)
        plt.plot([x for x in range(len(bMod))], bMod)
    else:
        plt.plot([x for x in range(len(ts1))], ts1)
        plt.plot([x for x in range(len(ts2))], ts2)
    plt.ylim(yMin, yMax)
    plt.show()

In [34]:
def TSDistanceMatrix(TSList, maxShiftPerc = 20, verbose = False):
    distMatrix = [[0 for y in range(len(TSList))] for x in range(len(TSList))]
    for i in range(len(TSList)):
        for j in range(i + 1, len(TSList)): 
            dist, shift = SWMatchEQ(TSList[i], TSList[j], maxShiftPerc=maxShiftPerc)
            distMatrix[i][j] = dist
        if verbose:
            print('{} of {} completed...\r'.format(i + 1, len(TSList)), end = '')
    return distMatrix

In [35]:
@lru_cache(maxsize=1024)
def clusterStringToIndexList(clusterString):
    """
    Converts (1,(2,3),(4,5)) -> [1,2,3,4,5]
    """
    return [int(x) for x in re.findall('(\d+)',clusterString)]

In [36]:
def aggloClusterAverage(TSList, numberOfClusters, maxShiftPerc = 20, verbose = False, measurement = 'shape'):
    """
    Performs agglomerative hierarchial clustering - distance between clusters is calculated
    as the average distance of every combination of time series within the clusters. I.E.
    given cluster (A,B) and cluster (C,D), the distance between them is calculated as:
    
            (dist(A,C) + dist(A,D) + dist(B,C) + dist(B,D)) / (2 * 2)
            
    Parameters:
        TSList: list of time series, parallel to the list countries
        maxShiftPerc: the percentage that time series are allowed to be shifted to
                      find the optimal matching
        distMatrix: distance matrix used to cluster on if one is already calculated
        verbose: Boolean set to whether the algorithm prints the clusters as they are formed
    """
    
    representativeList = [x for x in range(len(TSList))]
    distanceList = [0 for x in range(len(TSList))]
    clusterVariances = []
    lengthsList = [1 for x in range(len(TSList))]
    
    while len(representativeList) != numberOfClusters:
        #print('\r{}'.format(len(representativeList)), end = '')
        minDist = math.inf
        minDisti = -1
        minDistj = -1
        for i in range(len(representativeList)):
            for j in range(i + 1, len(representativeList)):
                clusterStringi = str(representativeList[i])
                clusterStringj = str(representativeList[j])
                
                distance = betweenClusterDist(clusterStringi, clusterStringj, measurement = measurement)
                
                if distance < minDist: 
                    minDist = distance
                    minDisti = i; minDistj = j
        
        if verbose:
            print((representativeList[minDisti], representativeList[minDistj]))
        
        representativeList.append((representativeList[minDisti], representativeList[minDistj]))
        representativeList.pop(minDistj)
        representativeList.pop(minDisti)
        
        lengthsList.append(lengthsList[minDistj] + lengthsList[minDisti])
        lengthsList.pop(minDistj)
        lengthsList.pop(minDisti)
        
        distanceList.append(minDist)
        distanceList.pop(minDistj)
        distanceList.pop(minDisti)
        
        clusterVariance = 0
        for i in range(len(lengthsList)):
            proportionTotalTS = lengthsList[i] / len(TSList)
            clusterVariance += proportionTotalTS * distanceList[i]
        
        clusterVariances.insert(0, clusterVariance)
        
    return representativeList, distanceList, clusterVariances

In [37]:
@lru_cache(maxsize=1024)
def betweenClusterDist(clusterStringi, clusterStringj, measurement = 'shape'):
    clusterListi = clusterStringToIndexList(clusterStringi)
    clusterListj = clusterStringToIndexList(clusterStringj)
    
    totalDist = 0
    for itemi in clusterListi:
        for itemj in clusterListj:
            if measurement == 'shape':
                totalDist += distMatrixNormalizedLocal[itemi][itemj]
            elif measurement == 'shape&raw':
                totalDist += distMatrixSizeShape[itemi][itemj]
            else:
                totalDist += distMatrix[itemi][itemj]
            
    distance = totalDist / (len(clusterListi) * len(clusterListj))
    return distance
    

## Computation

In [38]:
def SRC(ts1, ts2):
    """
    Calculates the spearman rank correlation between two ranked time series
    
    Parameters:
        ts1: first time series
        ts2: second time series
    """
    totalDiff = 0
    for i in range(len(ts1)):
        totalDiff += (ts1[i]-ts2[i])**2
    return 1 - ((6 * totalDiff) / (len(ts1) * (len(ts1)**2 - 1)))

def scaleAndInvertSRC(src):
    """
    Takes a spearman rank correlation value and normalizes it so that 0 is a perfect correlation
    and 1 is perfect negative correlation.
    
    Parameters:
        src: the spearman rank correlation value to normalize
    """
    
    return ((src * -1) + 1) / 2

In [39]:
distMatrixRanked = [[0 for x in range(len(rankedNewCaseTimeSeries))] for y in range(len(rankedNewCaseTimeSeries))]

In [40]:
for i in range(len(minRankedNewCaseTimeSeries)):
    for j in range(i + 1, len(minRankedNewCaseTimeSeries)):
        dist = SRC(minRankedNewCaseTimeSeries[i], minRankedNewCaseTimeSeries[j])
        dist = scaleAndInvertSRC(dist)
        distMatrixRanked[i][j] = dist
        distMatrixRanked[j][i] = dist
    print('\rCompleted: {} of {}'.format(i + 1, len(minRankedNewCaseTimeSeries)), end = '')

Completed: 157 of 157

In [41]:
for i in range(len(distMatrixRanked)):
    for j in range(i + 1, len(distMatrixRanked)):
        distMatrixRanked[j][i] = distMatrixRanked[i][j]
        
distMatrixRankedDF = pd.DataFrame(distMatrixRanked)
distMatrixRankedDF.columns = countries
distMatrixRankedDF.index = countries

distMatrixRankedDF.to_csv('distance_matrix_min_ranked_normalized.csv')

In [25]:
distMatrixRankedDF = pd.read_csv('Data/distance_matrix_ranked_normalized.csv', index_col=0)
distMatrixRanked = distMatrixRankedDF.values.tolist()

In [26]:
distMatrixRankedDF[['Japan','Tanzania']].loc[['Japan', 'Tanzania']]

,Japan,Tanzania
Japan,0.000000,0.852049
Tanzania,0.852049,0.000000


In [27]:
def plotCluster(TSL, clusterString, yMin = -3, yMax = 3):
    group = clusterStringToIndexList(str(clusterString))
    for index in group:
        plt.plot([x for x in range(len(TSL[index]))], TSL[index], color = 'silver', alpha = 0.4)
    avgTS = averageTS([TSL[x] for x in group])
    plt.plot([x for x in range(len(avgTS))], avgTS, color = 'violet')
    plt.ylim(yMin, yMax)
    plt.show()

In [29]:
with open(r'countries.txt', 'w') as fp:
    for item in countries:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [32]:
with open('rankedTS.txt', 'r') as fp:
    for line in fp:
        test.append(ast.literal_eval(line))